## Побудова POS-таггера методами машинного навчання.

У цьому ноутбуці буде використано два таггери: один побудований власноруч, на основі логістичної регресії; інший - адаптований із коду Метью Хонібала (https://github.com/sloria/textblob-aptagger/tree/master), на основі усередненого перцептрона. Обидві моделі дали схожі результати після тренування та тестування на відповідних вибірках з українського корпусу універсальних залежностей.

In [1]:
import conllu
import random
from collections import OrderedDict, Counter
import string
import gzip

from tokenize_uk import tokenize_words
from ukr_stemmer3 import UkrainianStemmer
import pymorphy2
morph = pymorphy2.MorphAnalyzer(lang='uk')

from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib

Для тренування використовувався train, для тюнингу ознак і параметрів моделі - dev-вибірка. Тут для тестування я використаю тільки test.

In [2]:
fname = 'uk_iu-ud-train.conllu.gz'
with gzip.open(fname, 'rb') as f:
    raw_train = f.read().decode()

fname2 = 'uk_iu-ud-test.conllu.gz'
with gzip.open(fname2, 'rb') as f2:
    raw_test = f2.read().decode()

In [3]:
train_set = conllu.parse(raw_train)
test_set = conllu.parse(raw_test)

(для фіч використано стеммер звідси: https://github.com/Amice13/ukr_stemmer)

In [4]:
class Tagger:
    """
    The POS-tagger based on logistic regression.
    """
    
    def __init__(self, load=True, model_loc='model.pkl'):
        self.MODEL_LOC = model_loc
        if load:
            self.MODEL = self.load(self.MODEL_LOC)
            
    def extract_features(self, i, sentence):
        """
        Feature extractor for the classifier.
        Sentence is a list of words.
        """
        stem = UkrainianStemmer(sentence[i]).stem_word()
        suf = 'NONE' if len(stem) == len(sentence[i]) else sentence[i][len(stem):]
        features = {
            'word': sentence[i],
            'word_stem': stem,
            'word_suf': suf,
            'prefix-3': sentence[i][:3],
            'suffix-3': sentence[i][-3:],
            'is_first': i == 0,
            'is_last': i == len(sentence) - 1,
            'has_hyphen': '-' in sentence[i],
            'is_numeric': sentence[i].isdigit(),
            'is_punct': sentence[i] in string.punctuation,
            'is_all_caps': sentence[i].upper() == sentence[i],
            'has_ascii': any(ord(char) < 128 for char in sentence[i])
        }
        if i == 0:
            features.update({
                'is_first': 1,
                'word-1': '<START>'
            })
        if i == len(sentence) - 1:
            features.update({
                'is_last': 1,
                'word+1': '<END>'
            })
        if i > 0:
            features.update({
                'is_capitalized': sentence[i][0].upper() == sentence[i][0],
                'word-1': sentence[i-1],
                'bigram-1': sentence[i-1]+'_'+sentence[i],
                'word-1-suf': sentence[i-1][-3:]
            })
        if i > 1:
            features.update({
                'word-2': sentence[i-2],
                'trigram-2': sentence[i-2] + '_' + sentence[i-1] + '_' + sentence[i],
                'word-2-suf': sentence[i-2][-3:]
            })
        if i < len(sentence) - 1:
            features.update({
                'word+1': sentence[i+1],
                'bigram+1': sentence[i]+'_'+sentence[i+1],
                'word+1-suf': sentence[i+1][-3:]
            })
        if i < len(sentence) - 2:
            features.update({
                'word+2': sentence[i+2],
                'trigram+2': sentence[i]+'_'+sentence[i+1]+'_'+sentence[i+2],
                'word+2-suf': sentence[i+2][-3:]
            })
        return features
    
    def process_corpus(self, corpus):
        """
        Get features and labels from the list of sentences,
        where each sentence is a list of tuples.
        """
        labels, features = [], []
        for sent in corpus:
            sent_words = [word[0] for word in sent]
            for i in range(len(sent)):
                labels.append(sent[i][1])
                feat_dict = self.extract_features(i, sent_words)
                # use previous tags in training
                if i > 0:
                    feat_dict.update({'pos-1': sent[i-1][1]})
                if i > 1:
                    feat_dict.update({'pos-2': sent[i-2][1]})
                features.append(feat_dict)
        return features, labels

    def get_features_unlabeled(self, corpus):
        """
        Get features from the list of sentences,
        where each sentence is a list of tokens.
        """
        features = []
        for sent in corpus:
            for i in range(len(sent)):
                feat_dict = self.extract_features(i, sent)
                features.append(feat_dict)
        return features
    
    def make_corpus(self, data):
        """
        Process data in CONLLU format.
        """
        corpus = []
        for sent in data:
            corpus.append([(w['form'], w['upostag']) for w in sent])
        return corpus
    
    def train_pipeline(self, train_features, train_labels, random_state=0, save=True):
        """
        Train a model for tagging.
        """
        vec = DictVectorizer()
        clf = LogisticRegression(random_state=random_state, penalty='l1')
        pipeline = Pipeline([('vec', vec), ('clf', clf)])
        pipeline.fit(train_features, train_labels)
        if save:
            self.save(pipeline)
        return pipeline
    
    def map_pos(self, word_parsed):
        """
        Map between pymorphy2 and UD POS tags.
        """
        MAPPING = {"ADJF": "ADJ", "ADJS": "ADJ", "COMP": "ADJ", "PRTF": "ADJ",
               "PRTS": "ADJ", "GRND": "VERB", "NUMR": "NUM", "ADVB": "ADV",
               "PRED": "ADV", "PREP": "ADP", "PRCL": "PART", "NOUN": "NOUN",
               "VERB": "VERB", "INTJ": "INTJ"}
        pm_tag = word_parsed.tag.POS
        if not pm_tag:
            return None
        if pm_tag == "CONJ":
            if "coord" in word_parsed.tag:
                pos = "CCONJ"
            else:
                pos = "SCONJ"
        else:
            pos = MAPPING.get(pm_tag, None)
        return pos
        
    def is_unambigous(self, word):
        """
        For use in cases where there is only one possible POS.
        """
        parsed_list = morph.parse(word)
        if len(parsed_list) == 1:
            return self.map_pos(parsed_list[0])
        elif len(set(word.tag.POS for word in parsed_list)) == 1:
            return self.map_pos(parsed_list[0])
        else:
            return None

    def classify_word(self, word_features, pipeline):
        """
        Classify word using both tagger and pymorphy2.
        """
        word = word_features['word']
        if 'is_capitalized' in word_features and word_features['is_capitalized']:
            return pipeline.predict([word_features])[0]
        pm_tag = self.is_unambigous(word)
        if pm_tag:
            return pm_tag
        else:
            return pipeline.predict([word_features])[0]

    def predict_tags(self, features, pipeline=None):
        """
        Make a prediction for a list of features.
        """
        if pipeline is None:
            pipeline = self.MODEL
        labels = []
        for i, f in enumerate(features):
            # use previous predicted tags 
            if 'word-1' in f and f['word-1'] != '<START>':
                f.update({'pos-1': labels[i-1]})
            if 'word-2' in f:
                f.update({'pos-2': labels[i-2]})
            labels.append(self.classify_word(f, pipeline))
        return labels
    
    def save(self, pipeline, fname='model.pkl'):
        """
        Save a fitted model for tagger.
        """
        joblib.dump(pipeline, fname)
        print('Model is saved to', fname)
        
    def load(self, fname='model.pkl'):
        """
        Load a pretrained model.
        """
        pipeline = joblib.load(fname)
        return pipeline
    
    def tag_sentence(self, sent, pipeline=None):
        """
        Tag a sentence with pretrained model.
        Returns a list of tuples (word, tag).
        """
        if pipeline is None:
            pipeline = self.MODEL
        sent_tokenized = tokenize_words(sent)
        sent_features = self.get_features_unlabeled([sent_tokenized])
        tags = self.predict_tags(sent_features, pipeline)
        return list(zip(sent_tokenized, tags))

In [5]:
t = Tagger(load=False)
train_corpus = t.make_corpus(train_set)
train_features, train_labels = t.process_corpus(train_corpus)
t.train_pipeline(train_features, train_labels, random_state=477)

Model is saved to model.pkl


Pipeline(memory=None,
     steps=[('vec', DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
        sparse=True)), ('clf', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=477, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [6]:
t = Tagger()
test_corpus = t.make_corpus(test_set)
test_features, test_labels = t.process_corpus(test_corpus)
pred_labels = t.predict_tags(test_features)
print(classification_report(test_labels, pred_labels, digits=3))

             precision    recall  f1-score   support

        ADJ      0.942     0.944     0.943      1723
        ADP      0.996     0.987     0.991      1377
        ADV      0.905     0.883     0.894       618
        AUX      0.846     0.206     0.331       107
      CCONJ      0.957     0.975     0.966       554
        DET      0.956     0.897     0.925       552
       INTJ      1.000     0.400     0.571        10
       NOUN      0.960     0.961     0.960      4021
        NUM      0.913     0.930     0.922       272
       PART      0.905     0.814     0.857       306
       PRON      0.900     0.910     0.905       423
      PROPN      0.802     0.852     0.827       576
      PUNCT      0.996     1.000     0.998      2677
      SCONJ      0.804     0.944     0.869       231
        SYM      1.000     0.375     0.545        16
       VERB      0.932     0.985     0.957      1359
          X      0.939     0.915     0.926       117

avg / total      0.949     0.949     0.947  

In [7]:
print(round(accuracy_score(test_labels, pred_labels), 3))

0.949


Точність таггера 95% - дуже непогано! Можна подивитись, де найбільше помиляється таггер:

In [8]:
conf_pairs = []
test_words = [f['word'] for f in test_features]
for w, l, p in zip(test_words, test_labels, pred_labels):
    if l != p:
        conf_pairs.append((l, p))

Counter(conf_pairs).most_common(10)

[(('NOUN', 'PROPN'), 88),
 (('AUX', 'VERB'), 78),
 (('PROPN', 'NOUN'), 54),
 (('NOUN', 'ADJ'), 51),
 (('DET', 'PRON'), 36),
 (('ADV', 'NOUN'), 36),
 (('PROPN', 'ADJ'), 27),
 (('ADJ', 'PROPN'), 24),
 (('ADJ', 'ADV'), 24),
 (('ADJ', 'NOUN'), 24)]

Плутанина між іменником та іменником-власною назвою - дуже зрозуміла, хоч і неприємна для перспектив NER. AUX i VERB плутати логічно, так само як DET i PRON, а ось плутанина між NOUN i ADJ менш прийнятна.

<hr>

Перцептрон має переваги порівняно з попередньою моделлю, насамперед у швидкості та розмірі натренованої моделі. Для таггера я додав пару ознак плюс перевірку за словником pymorphy2 (тестування на реченнях, несхожих на тренувальний корпус, показало обмеженість натренованого словника перцептрона).

In [9]:
from perceptron_tagger.tagger import PerceptronTagger

def perceptron_train_and_save(train_corpus, nr_iter=10, 
                              fname='uk_perceptron_tagger.pickle'):
    perc_train = []
    for sent in train_corpus:
        words = [w[0] for w in sent]
        tags = [w[1] for w in sent]
        perc_train.append((words, tags))
    p = PerceptronTagger(load=False)
    p.train(perc_train, nr_iter=nr_iter, save_loc=fname)
    
print('Training and saving the perceptron POS tagger.')
perceptron_train_and_save(train_corpus, 20)
print('Done!')

Training and saving the perceptron POS tagger.
Done!


In [10]:
def process_test_set(test_set):
    sents = ''
    for sent in test_set:
        sent = ' '.join([w['form'].replace(' ', '') for w in sent])
        sents += sent.strip() + '\n'
    return sents

test_perc = process_test_set(test_set)
pt = PerceptronTagger()
perc_pred_labels = [word[1] for word in pt.tag(test_perc, tokenize=False)]
print(classification_report(test_labels, perc_pred_labels, digits=3))

             precision    recall  f1-score   support

        ADJ      0.937     0.954     0.945      1723
        ADP      0.995     0.987     0.991      1377
        ADV      0.908     0.909     0.909       618
        AUX      0.815     0.206     0.328       107
      CCONJ      0.962     0.960     0.961       554
        DET      0.944     0.850     0.894       552
       INTJ      0.500     0.300     0.375        10
       NOUN      0.953     0.964     0.959      4021
        NUM      0.969     0.912     0.939       272
       PART      0.853     0.797     0.824       306
       PRON      0.894     0.898     0.896       423
      PROPN      0.833     0.875     0.854       576
      PUNCT      0.998     0.999     0.998      2677
      SCONJ      0.818     0.935     0.873       231
        SYM      1.000     0.375     0.545        16
       VERB      0.933     0.985     0.958      1359
          X      0.958     0.786     0.864       117

avg / total      0.948     0.948     0.946  

In [11]:
print(round(accuracy_score(test_labels, perc_pred_labels), 3))

0.948


Цей таггер менш точний, але його натренована модель займає всього 4 мегабайти (порівняно з 60 мб для логістичної регресії).

Приклад на несхожому тексті (зі свіжих новин): порівняємо наші таггери і pymorphy2. (Я довго шукав такий текст, на якому були б хоч якісь грубі помилки таггерів, але поки не знайшов; є проблема зі словами, у яких є дефіс, але це проблема токенізації).

In [12]:
text = """
Уявімо, що завдання науковця - не зробити відкриття, а створити скульптуру.
Будь-яку і з будь-чого. 
Хороші вчені, які бажають співпрацювати із закордоном, бути визнаними у світі, пишатися своїм витвором - докладатимуть зусиль, створюючи витончені форми з мармуру, бронзи чи, принаймні, гіпсу.
"""
sentences = [sent.strip(' \n') for sent in text.split('\n') if not sent == '']
for sent in sentences:
    tagged_sent = t.tag_sentence(sent)
    perc_tagged_sent = pt.tag(sent)
    words = [w[0] for w in tagged_sent]
    tags1 = [w[1] for w in tagged_sent]
    tags2 = [w[1] for w in perc_tagged_sent]
    pytags = [morph.parse(w)[0].tag.POS for w in words]
    for word, tag1, tag2, pytag in zip(words, tags1, tags2, pytags):
        print(word, tag1, tag2, pytag)

Уявімо VERB VERB VERB
, PUNCT PUNCT None
що SCONJ SCONJ CONJ
завдання NOUN NOUN NOUN
науковця NOUN NOUN NOUN
- PUNCT PUNCT None
не PART PART PRCL
зробити VERB VERB VERB
відкриття NOUN NOUN NOUN
, PUNCT PUNCT None
а CCONJ CCONJ CONJ
створити VERB VERB VERB
скульптуру NOUN NOUN NOUN
. PUNCT PUNCT None
Будь VERB VERB VERB
- PUNCT PUNCT None
яку DET DET NPRO
і CCONJ CCONJ CONJ
з ADP ADP PREP
будь VERB VERB VERB
- PUNCT PUNCT None
чого PRON PRON NPRO
. PUNCT PUNCT None
Хороші ADJ ADJ ADJF
вчені NOUN NOUN ADJF
, PUNCT PUNCT None
які DET DET NPRO
бажають VERB VERB VERB
співпрацювати VERB VERB VERB
із ADP ADP PREP
закордоном NOUN NOUN NOUN
, PUNCT PUNCT None
бути AUX AUX NOUN
визнаними ADJ ADJ ADJF
у ADP ADP PREP
світі NOUN NOUN NOUN
, PUNCT PUNCT None
пишатися VERB VERB VERB
своїм DET DET NPRO
витвором NOUN NOUN NOUN
- PUNCT PUNCT None
докладатимуть VERB VERB VERB
зусиль NOUN NOUN NOUN
, PUNCT PUNCT None
створюючи VERB VERB GRND
витончені ADJ ADJ ADJF
форми NOUN NOUN NOUN
з ADP ADP PREP
марму

<hr>
Для багатьох задач такий поділ на частини мови не потрібний: AUX i VERB, DET i PRON, SYM i PUNCT можуть бути взаємозамінними. Можна подивитись на точність таггерів у такому випадку.

In [13]:
def combine_tags(tag_list):
    mapping = {'DET': 'PRON', 'SYM': 'PUNCT', 
               'AUX': 'VERB', 'PROPN': 'NOUN'}
    return [mapping.get(tag, tag) for tag in tag_list]

print(round(accuracy_score(combine_tags(test_labels), combine_tags(pred_labels)), 3))

0.968


In [14]:
print(round(accuracy_score(combine_tags(test_labels), combine_tags(perc_pred_labels)), 3))

0.967


Покращення порівняно з бейзлайном (86%-89%) очевидне.